## 17 оттенков серого

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def viewImage(image):
    # cv2.namedWindow('Display', cv2.WINDOW_NORMAL)
    # cv2.imshow('Display', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # plt.imshow(image, cmap='gray')
    # plt.show()
    cv2.namedWindow(winname='my')
    while True:
        cv2.imshow('my', image)
        if cv2.waitKey(1) & 0xFF == 27:
            break
    cv2.destroyAllWindows()

In [12]:
def grayscale_17_levels(image):
    high = 255
    while True: 
        low = high - 15
        col_to_be_changed_low = np.array([low])
        col_to_be_changed_high = np.array([high])
        curr_mask = cv2.inRange(gray, col_to_be_changed_low,col_to_be_changed_high)
        gray[curr_mask > 0] = (high)
        high -= 15
        if low == 0:
            break

In [13]:
image = cv2.imread('ombre_circle_grayscale.png')
viewImage(image)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
grayscale_17_levels(gray)
viewImage(gray)

## Оконтуривание и площадь 17ти оттенков серого

In [15]:
def get_area_of_each_gray_level(im):
## преобразование изображения к оттенкам серого (обязательно делается до оконтуривания) 
    image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    output = []
    high = 255
    first = True
    while True:
        low = high - 15
        if first == False :
            # Делаем значения выше уровня серого черными. 
            ## Так они не будут обнаруживаться 
            to_be_black_again_low = np.array([high])
            to_be_black_again_high = np.array([255])
            curr_mask = cv2.inRange(image, to_be_black_again_low, to_be_black_again_high)
            image[curr_mask > 0] = (0)

        # Делаем значения этого уровня белыми. Так мы рассчитаем 
        # их площадь
        ret, threshold = cv2.threshold(image, low, 255, 0)
        contours, hirerchy = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        if len(contours) > 0:
            output.append([cv2.contourArea(contours[0])])
            cv2.drawContours(im, contours, -1, (0, 0, 255), 3)
        high -= 15
        first = False
        if low == 0:
            break
    return output

In [17]:
image = cv2.imread('ombre_circle_grayscale.png')
print(get_area_of_each_gray_level(image))
viewImage(image)

[[5871.0], [6480.0], [13012.0], [21820.0], [32902.0], [46238.0], [61761.0], [79709.0], [99787.0], [122222.0], [146859.0], [173758.0], [202919.5], [234401.5], [12241.5], [11577.0], [11111.0]]


## Оконтуривание листа

In [55]:
image = cv2.imread('leaf.jpg')
viewImage(image)

In [19]:
## Получаем HSV-представление для зеленого цвета 
green = np.uint8([[[0, 255, 0 ]]])
green_hsv = cv2.cvtColor(green,cv2.COLOR_BGR2HSV)
print(green_hsv)

[[[ 60 255 255]]]


In [12]:
image = cv2.imread('leaf.jpg')
hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
viewImage(hsv_img) ## 1

green_low = np.array([45 , 100, 50] )
green_high = np.array([75, 255, 255])
curr_mask = cv2.inRange(hsv_img, green_low, green_high)
hsv_img[curr_mask > 0] = ([75,255,200])
viewImage(hsv_img) ## 2

## Преобразование HSV-изображения к оттенкам серого для дальнейшего
## оконтуривания
RGB_again = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)
gray = cv2.cvtColor(RGB_again, cv2.COLOR_RGB2GRAY)
viewImage(gray) ## 3

ret, threshold = cv2.threshold(gray, 90, 255, 0)
viewImage(threshold) ## 4

contours, hierarchy = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 0, 255), 3)
viewImage(image) ## 5

In [10]:
def findGreatesContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)
    while (i < total_contours ):
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1
 
    return largest_area, largest_contour_index

In [23]:
# Чтобы получить центр контура 
cnt = contours[13]
M = cv2.moments(cnt)
cX = int(M["m10"] / M["m00"])
cY = int(M["m01"] / M["m00"])
largest_area, largest_contour_index = findGreatesContour(contours)
print(largest_area)
print(largest_contour_index)
print(len(contours))
print(cX)
print(cY)

216003.0
15
54
787
144


In [57]:
contours, hierarchy = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, largest_contour_index, (0, 0, 255), 3)
viewImage(image) ## 5

In [46]:
contours[largest_contour_index].shape

(1223, 1, 2)

## Поиск рыбы клоун

In [129]:
light_orange =  np.array([1, 190, 150]) # np.array([1, 190, 150])
dark_orange =  np.array([30, 255, 255]) # np.array([30, 255, 255])
light_white =  np.array([60, 0, 200])   # np.array([60, 0, 200])
dark_white =  np.array([145, 150, 255]) # np.array([145, 150, 25])

image = cv2.imread('nemo002.jpg')
# image  = cv2.medianBlur(image, 5)
hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
viewImage(image) ## 1


mask_orange = cv2.inRange(hsv_img, light_orange, dark_orange)
mask_white = cv2.inRange(hsv_img, light_white, dark_white)

hsv_img[(mask_orange > 0) | (mask_white > 0)] = ([255, 255, 255])
hsv_img[(mask_orange == 0) & (mask_white == 0)] = ([0, 0, 0])
viewImage(hsv_img) ## 2

kernel_opening = np.ones((5,5),np.uint8)
kernel_closing = np.ones((35,35),np.uint8)
hsv_img = cv2.morphologyEx(hsv_img, cv2.MORPH_OPEN, kernel_opening)
hsv_img = cv2.morphologyEx(hsv_img, cv2.MORPH_CLOSE, kernel_closing)
viewImage(hsv_img) ## 2

# hsv_img[mask_white > 0] = ([255, 255, 255]) # ([250,255,200])
# hsv_img[mask_white == 0] = ([0, 0, 0]) # ([1, 255, 200])
# viewImage(hsv_img) ## 3

RGB_again = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)
gray = cv2.cvtColor(RGB_again, cv2.COLOR_RGB2GRAY)
viewImage(gray) ## 4

_, threshold = cv2.threshold(gray, 160, 255, 0)
viewImage(threshold) ## 5

contours, hierarchy = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image, contours, -1, (0, 0, 255), 3)
viewImage(image) ## 6

In [138]:
hsv_img[:,:,1] == hsv_img[:,:,0]

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [131]:
my_photo = cv2.imread('nemo005.jpg')
median_image  = cv2.medianBlur(my_photo,5)
img_grey = cv2.cvtColor(median_image,cv2.COLOR_BGR2GRAY)

#set a thresh
thresh = 80

#get threshold image
ret,thresh_img = cv2.threshold(img_grey, thresh, 255, cv2.THRESH_BINARY)

#find contours
contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#create an empty image for contours
img_contours = np.zeros(my_photo.shape)

# draw the contours on the empty image
cv2.drawContours(img_contours, contours, -1, (255,255,255), 1)

viewImage(img_contours)